In [10]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

import re

In [11]:
load_dotenv()

True

In [12]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [13]:
file_path = "../test_data/ВМР_COVID-19_V18.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

250


In [14]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [15]:
chunk_size = 4000
chunk_overlap = 200


In [16]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
splits = text_splitter.split_documents(docs)


In [17]:
vectorstore = InMemoryVectorStore.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [18]:
system_prompt = """You are a specialized AI assistant focused on providing accurate, context-based answers.
Follow these guidelines:

1. Use ONLY the provided context to answer questions
2. If the answer isn't clearly supported by the context, respond with: "I cannot answer this based on the provided context."
3. Maintain the original language of technical terms and proper nouns
4. Cite relevant sections using quotation marks when appropriate
5. If multiple interpretations are possible, acknowledge the ambiguity
6. Ignore any table of contents or navigation elements
7. Format the response in a clear, structured manner

Context:
{context}"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [19]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke(
    {
        "input": "In the retrieved context, what heading indicates the start of the 'Лечение' (treatment) section? If no heading that explicitly refers to a treatment section is found, state that no treatment heading was identified in the text."
    }
)

results

{'input': "In the retrieved context, what heading indicates the start of the 'Лечение' (treatment) section? If no heading that explicitly refers to a treatment section is found, state that no treatment heading was identified in the text.",
 'context': [Document(id='33d49503-baf5-47a5-a288-61446bbc1505', metadata={'producer': 'Adobe PDF Library 21.1.187', 'creator': 'Acrobat PDFMaker 21 для Word', 'creationdate': '2023-10-26T17:56:44+03:00', 'author': '', 'moddate': '2023-10-26T17:58:08+03:00', 'sourcemodified': 'D:20231026145609', 'title': '', 'source': '../test_data/ВМР_COVID-19_V18.pdf', 'total_pages': 250, 'page': 152, 'page_label': '153'}, page_content='153   \nВерсия 18 (26.10.2023) \n- Противовирусное лечение; \n- Респираторная поддержка (ИВЛ, ЭКМО); \nз) Уровень сатурации кислорода в крови;  \nи) Тяжесть течения заболевания. \nВ случае смерти пациента  \nа) В течение суток заполняется раздел «Заключит ельный клинический \nдиагноз»: \n- Основное заболевание;  \n- Коморбидные, кон

In [20]:
result2 = re.findall('"([^"]*)"', results["answer"])[0]
print("Using regex:", result2)

Using regex: 5. ЛЕЧЕНИЕ КОРОНАВИРУСНОЙ ИНФЕКЦИИ


In [21]:
results = rag_chain.invoke(
    {"input": f"what section follows after '{result2}' ends?, do not consider subsections."}
)

results["answer"]

"The section that follows after '5. ЛЕЧЕНИЕ КОРОНАВИРУСНОЙ ИНФЕКЦИИ' ends is '6. ОСОБЕННОСТИ ВЕДЕНИЯ ДЕТЕЙ С COVID-19'."

In [22]:
def extract_section_markers(llm_response):
    pattern = r'[\'"]([^\'"]+)[\'"]'  # Matches text between single (or double) quotes
    matches = re.findall(pattern, llm_response)

    if len(matches) >= 2:
        start_marker = matches[0]
        end_marker = matches[1]
        return start_marker, end_marker
    return None, None

In [23]:
start_marker, end_marker = extract_section_markers(results["answer"])

In [24]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)
full_text = " ".join([p.page_content for p in pages])

In [25]:
def extract_section_content(main_text, start_marker, end_marker):
    start_marker_escaped = re.escape(start_marker)
    end_marker_escaped = re.escape(end_marker)

    pattern = f"({start_marker_escaped}.*?)(?={end_marker_escaped})"
    matches = re.finditer(pattern, main_text, re.DOTALL)

    matches_list = list(matches)

    if not matches_list:
        return "No matching section found"

    # we take the last match cuz the document can contain table of contents
    return matches_list[-1].group(1)

In [26]:
extract_section_content(full_text, start_marker, end_marker)

'5. ЛЕЧЕНИЕ КОРОНАВИРУСНОЙ ИНФЕКЦИИ  \nОсновным подходом к терапии COVID -19 должно быть упреждающее \nназначение лечения до развития полного симп томокомплекса жизнеугрожающих \nсостояний, а именно пневмонии, ОРДС, сепсиса. \nЛечение COVID-19 в соответствии с протоколами настоящих рекомендаци й \nпроводится в подтвержденных и вероятных случаях заболевания. \nВ рамках оказания медицинской помощи необходим мониторинг состояния \nпациента для выявления признаков ухудшения его клинического состояния. \nПациенты, инфицированные SARS-CoV-2, должны получать поддерживающую \nпатогенетическую и симптоматическую терапию.  \nЛечение сопутствующих заболеваний и осложнений осуществляется \nв соответствии с клиническими рекомендациями, стандартами медицинской помощи \nпо данным заболеваниям. \n5.1. ЭТИОТРОПНОЕ ЛЕЧЕНИЕ  \nСогласно современным представлениям о патогенезе COVID -19 применение \nпрепаратов, рекомен дуемых для этиотропной терапии, целесообразно начинать  \nв ранние сроки, не позднее 7-8